### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
int *h_a, *h_b, *h_c;

In [ ]:
size_t bytes = sizeof(int) * n;

In [ ]:
h_a = (int*)malloc(bytes)

In [2]:
import torch.distributed as dist
import torch.distributed.rpc as rpc

In [ ]:
class TensorParallelGroupInitializer(ProcessGroup):
    def init_process_group(self):
        num_tensor_parallel_groups = self.world_size // self.tensor_parallel_size
        
        for i in range(num_tensor_parallel_groups):
            ranks = list(range(
                i*self.tensor_parallel_size,
                (i+1)*self.tensor_parallel_size
            ))
            process_group = dist.new_group(ranks=ranks)
            
            if self.rank in ranks:
                local_rank = ranks.index(self.rank)
                local_world_size = len(world_size)

In [ ]:
def free_storage(x):
    x.storage().resize_(0)

In [ ]:
step 1: mask targets
step 2: calculate local predicted logits
step 3: calcualte global predicted logits
step 4: calculate log(...)
step 5: calculate loss

In [3]:
n_microbatches = 4
n_partitions = 3

In [4]:
n_clock_cycles = n_microbatches+n_partitions-1

In [6]:
for clock_idx in range(n_clock_cycles):
    start_partition_idx = max(clock_idx+1-n_microbatches, 0)
    end_partition_idx = min(clock_idx+1, n_partitions)
    
    xs = []
    for partition_idx in range(start_partition_idx, end_partition_idx):
        microbatch_idx = clock_idx - partition_idx
        xs.append([microbatch_idx, partition_idx])
        
    print(xs)

[[0, 0]]
[[1, 0], [0, 1]]
[[2, 0], [1, 1], [0, 2]]
[[3, 0], [2, 1], [1, 2]]
[[3, 1], [2, 2]]
[[3, 2]]


In [ ]:
for group in groups:
    dist.barrier()
    dist.destroy_process_group()

In [ ]:
step 1: ln1 = ln1(resid_pre)
step 2: attn_out = attn1(ln1) + attn2(ln1) + ...
step 3: mid_resid = resid_pre + attn_out
step 4: ln2 = ln2(mid_resid)
step 5: mlp = mlp(ln2)
step 6: final_resid = mlp + mid_resid

In [ ]:
def compute

In [ ]:
W_pos = model.W_pos

similarity = torch.cosine_similarity(W_pos[:, 0], W_pos[:, 1])

In [ ]:
MLP(Attn(x @ W_E )) @ W_U

In [ ]:
logits, _ = model.run_with_cache(past_moves)

In [ ]:
log_probs = F.log_softmax(logits[:, -1, :], dim=-1)

In [ ]:
board_state = torch.zeros(board_size, board_size)
board_state.view(-1)[:] = log_probs

In [ ]:
int *h_a, *h_b, *h_c;

In [ ]:
size_t bytes = (int*)sizeof(int)

In [ ]:
h_a = malloc(bytes)

In [12]:
from einops import rearrange

In [13]:
class _ParallelCrossEntropy(torch.autograd.Function):
    @staticmethod
    def forward(ctx, parallel_logits, targets, parallel_context):
        # calculate predicted
        rank = parallel_context.get_local_rank(ParallelMode.TENSOR)
        partition_size = parallel_logits.shape[-1]
        vocab_start_idx = partition_size*rank
        vocab_end_idx = vocab_start_idx+partition_size
        
        target_mask = (targets < vocab_start_idx) | (targets >= vocab_end_idx)
        masked_targets = targets.clone() - vocab_start_idx
        masked_targets[target_mask] = 0.
        
        parallel_logits = rearrange(
            parallel_logits,
            "batch_size seq_len vocab_size -> (batch_size seq_len) vocab_size"
        )
        targets_1d = rearrange(
            targets,
            "batch_size seq_len -> (batch_size seq_len)"
        )
        predicted_logits = parallel_logits[
            torch.arange(targets.shape[-1]),
            targets_1d
        ]
        predicted_logits = all_reduce(
            predicted_logits,
            paralllel_context=parallel_context,
            parallel_mode=ParallelMode.TENSOR
        )
        
        exp_logits = torch.exp(parallel_logits)
        exp_logits = all_reduce(exp_logits)
        loss = exp_logits.sum(dim=-1).log() - predicted_logits
        return loss

In [9]:
class ParallelCrossEntropy(nn.Module):
    def __init__(self, parallel_context):
        super().__init__()
        self.parallel_context = parallel_context
    
    def forward(self, logits, targets):
        loss = _ParallelCrossEntropy.apply(
            logits, targets
        )
        
        loss /= len(targets)
        
        return loss

In [ ]:
ready
begin forward
finished forward
finished backward
finished batch

In [ ]:
handshake
syncronization

In [ ]:
int *h_a, *h_b, *h_c;

In [ ]:
size_t bytes = sizeof(int)*n

In [ ]:
h_a = (int*)malloc(bytes);
h_b = (int*)malloc(bytes);
h_c = (int*)malloc(bytes);

In [ ]:
_, clean_cache = model.run_with_cache(clean_tokens)
_, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [14]:
def patch_head(acts, hook, clean_cache, corrupted_cache, target_head):
    target_layer_idx, target_head_idx = target_head
    
    if hook.layer() == target_layer_idx:
        acts[:, :, head_idx] = corrupted_cache[hook.name][:, :, head_idx]
    else:
        acts = clean_cache[hook.name]
    
    return acts

In [17]:
from transformer_lens.utils import get_act_name
from functools import partial

In [ ]:
results = torch.zeros(n_layers, n_heads)

for layer_idx in range(n_layers):
    for head_idx in range(n_heads):
        hook_name = get_act_name("z", layer_idx)
        hook_func = partial(
            patch_head,
            clean_cache=clean_cache,
            corrupted_cache=corrupted_cache,
            target_head=(layer_idx, head_idx)
        )
        patched_logits, _ = model.run_with_cache(
            clean_tokens,
            fwd_hooks=[(hook_name, hook_func)]
        )
        results[layer_idx, head_idx] = compute_ioi_metric(patched_logits)

In [18]:
layer_idx = 3

In [ ]:
_, cache = model.run_with_cache(past_moves)

In [ ]:
top_neurons = cache["post", layer_idx].std(dim=[0, 1]).argsort(descending=True)

In [ ]:
W_E = model.W_E
W_V = model.W_V[layer_idx, head_idx]
W_O = model.W_O[layer_idx, head_idx]
W_U = model.W_U

In [ ]:
W_E @ W_V @ W_O @ W_E.T

In [ ]:
xs = W_A[:, :, 0]
ys = W_A[:, :, 1 ]

In [19]:
def intervene_feature(resid, feature, scale):
    feature /= feature.norm(dim=-1)
    feature_projection = resid[0, position] @ feature
    
    resid[0, position] = 

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
cache["embed"][:, -1] @ W_U[:, tokens[1:]]

In [20]:
def probabiliy_scores(image_embedding, text_embedding):
    image_embedding /= image_embedding.norm(dim=-1)
    text_embedding /= text_embedding.norm(dim=-1)
    
    similarities = image_embedding @ text_embedding.T
    probs = F.softmax(similarities, dim=-1)
    return probs

In [ ]:
discount_returns = discounted_return_at_each_step(rewards)

In [ ]:
total_loss = 0

for discount_return, prob in zip(discount_returns, selected_action_probs):
    pass

In [ ]:
for _ in range(N_EPISODES):
    state, _ = env.reset
    state = torch.from_numpy(state) # ignore
    done = False
    
    while done:
        predicted_rewards = model(state)
        action = torch.argmax(predicted_rewards, dim=-1)
        action = action.item() # ignore
        
        state, reward, done, _, _ = env.step(action)
        state = torch.from_numpy(state) # ignore
        
        if done is True:
            target_reward = torch.tensor(reward)
        else:
            with torch.no_grad():
                predicted_next_rewards = model(state)
            
            predicted_next_reward = torch.max(
                predicted_next_rewards, dim=-1
            )
            predicted_next_reward = predicted_next_reward[0] # ignore
            target_reward = reward + GAMMA * predicted_next_reward
        
        predicted_reward = predicted_rewards[action]
        loss = loss_func(predicted_rewards[action], target_reward)
        loss.append(loss.detach().numpy()) # ignore
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
W_E = model.W_E
W_V = model.W_V[layer_idx, head_idx]
W_O = model.W_O[layer_idx, head_idx]
W_U = model.W_U

In [ ]:
full_OV_circuit = W_E @ W_V @ W_O @ W_U

In [ ]:
_, cache = model.run_with_cache(past_moves)

In [ ]:
top_neurons = cache["post", layer_idx].std(dim=[0, 1]).argsort(descending=True)

In [21]:
from einops import einsum

In [ ]:
def compute_consine_similarity(neuron_idx, feature):
    feature /= feature.norm(dim=-1)
    W_out = model.W_out[layer_idx, neuron_idx]
    W_out /= W_out.norm(dim=-1)
    
    return einsum(
        feature,
        W_out
    )

In [ ]:
tokens = model.to_tokens(text)

embed = model.embed
mlp0 = model.blocks[0].mlp
ln1 = model.blocks[0].ln1

In [ ]:
embeddings = embed(tokens)

resid_after_mlp0 = embeddings + mlp0(ln1(embeddings) )

step 1: a clean prompt, and a corrupted prompt
step 2: record all the interdimate activations of the clean prompt, and the corrupted prompt
step 3: patch the activations from the corrupted prompt if a component is a part of the direct path, otherwise patch clean activations
step

In [ ]:
record, p,

In [22]:
layer_idx, head_idx = 5, 1393

In [ ]:
_, cache = model.run_with_cache(board_states)

In [ ]:
mlp_acts = cache["post", layer_idx][:, neuron_idx]
threshold = mlp_acts.quantile(0.99)
top_neurons = mlp_acts > threshold

In [ ]:
(board_states == 1)

In [ ]:
softmax(x @ W_Q @ W_K.T @ x) @ x @ W_V @W_O 

In [23]:
def compute_metric(pattern, target_pattern):
    return (pattern * target_pattern).sum() / (pattern.sum())

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
data = torch.zeros(n_layers, n_heads)
for layer_idx in range(n_layers):
    for head_idx in range(n_heads):
        hook_name = f"blocks.{layer_idx}.attn.hook_pattern"
        pattern = cache[hook_name][:, head_idx]
        score = compute_metric(pattern, target_pattern)
        data[layer_idx, head_idx] = score

In [ ]:
deploy on edge
database
server
service

In [ ]:
record, memory, process, send

In [ ]:
population activity, motor imagery, stimulus, conditioned

In [ ]:
neuron1 > axon > synapse >  dendiret>   neuron2 

- Step 1: The output of head 0.0 is $h(x) = \frac{1}{n} \sum_{i=1}^n x_i^T W_{OV}^{0.0}$. This is a weighted average of the input vectors $x_i$ transformed by the OV matrix.

- Step 2: We can write each input vector as $x_j = \text{pos}_j + \text{tok}_j$. The vector has a positional encoding component and a token embedding component.

- Step 3: Plugging this in gives: $h(x) = \frac{1}{n}(\sum_{i=1}^n \text{pos}_i^T W_{OV}^{0.0} + \sum_{i=1}^n \text{tok}_i^T W_{OV}^{0.0})$

- Step 4: The token embeddings consist of left brackets and right brackets. Let $n_L$ and $n_R$ be the number of each token type.
+ $v_L = n_L * tok(\text{left bracket}) * W_{OV}^{0, 0}$
+ $v_R = n_R * tok(\text{right bracket}) * W_{OV}^{0, 0}$

- Step 5: This gives the final formula: $h(x) = \frac{1}{n}(\sum_{i=1}^n \text{pos}_i^T W_{OV}^{0.0} + n_L v_L + n_R v_R)$

In [43]:
xs = torch.randn(5)

In [44]:
global_max = 2.1

In [54]:
import torch
import torch.distributed as dist

Calculate the global maximum value of `xs`, given that `xs` is the current partition on the current GPU

In [55]:
xs

tensor([-0.8253,  1.3883,  0.7739, -1.5190, -1.1593])

In [56]:
local_max = torch.max(xs, dim=-1)[0]

In [57]:
local_max

tensor(1.3883)

In [ ]:
global_max = dist.all_reduce(
    local_max,
    op=dist.ReduceOp.MAX,
)

In [58]:
normalized_xs = xs - global_max

In [59]:
normalized_xs

tensor([-2.9253, -0.7117, -1.3261, -3.6190, -3.2593])

In [88]:
x = torch.arange(0, 6).view((2, 3))
y = torch.arange(0, 6).view((3, 2))

In [112]:
import torch

In [113]:
x.shape, y.shape

(torch.Size([2, 3]), torch.Size([3, 2]))

Write the manual version of einsum below

In [114]:
output = torch.zeros(x.shape[0], y.shape[1])
output.shape

torch.Size([2, 2])

In [115]:
for i in range(x.shape[0]):
    for j in range(y.shape[1]):
        total = 0
        for k in range(3):
            total += x[i, k] * y[k, j]
        
        output[i, j] = total

In [116]:
output

tensor([[10., 13.],
        [28., 40.]])

In [117]:
output == torch.einsum("ik, kj -> ij", x, y)

tensor([[True, True],
        [True, True]])

In [121]:
in_queues = []

In [123]:
in_queues.append(1)

In [124]:
in_queues

[1]

In [125]:
in_queues.append(2)

In [126]:
in_queues

[1, 2]